<a href="https://colab.research.google.com/github/robertoarturomc/ProgramacionConcurrente/blob/main/27_Machine_Learning_Spark_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Programación Concurrente
## 27. Machine Learning con Spark I

Para ver cómo era la versión "Viejita" (con `spark.mllib`), revisa este blog:

https://medium.com/data-science-school/practical-apache-spark-in-10-minutes-part-4-mllib-fca02fecf5b8


In [ ]:
import pandas as pd
import numpy as np

from sklearn.datasets import load_iris

from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [ ]:
data = load_iris()
cols = [i.replace('(cm)','').strip().replace(' ','_') for i in data.feature_names] + ['species'] # Limpieza de los nombres de las columnas
pdf = pd.DataFrame(np.c_[data.data, data.target], columns=cols)
df = spark.createDataFrame(pdf)
df.show()

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|species|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2|    0.0|
|         4.9|        3.0|         1.4|        0.2|    0.0|
|         4.7|        3.2|         1.3|        0.2|    0.0|
|         4.6|        3.1|         1.5|        0.2|    0.0|
|         5.0|        3.6|         1.4|        0.2|    0.0|
|         5.4|        3.9|         1.7|        0.4|    0.0|
|         4.6|        3.4|         1.4|        0.3|    0.0|
|         5.0|        3.4|         1.5|        0.2|    0.0|
|         4.4|        2.9|         1.4|        0.2|    0.0|
|         4.9|        3.1|         1.5|        0.1|    0.0|
|         5.4|        3.7|         1.5|        0.2|    0.0|
|         4.8|        3.4|         1.6|        0.2|    0.0|
|         4.8|        3.0|         1.4|        0.1|    0.0|
|         4.3|        3.0|         1.1| 

In [ ]:
# Preparamos nuestras variables objetivo y de entrenamiento.

label_indexer = StringIndexer(inputCol="species", outputCol="label")
assembler = VectorAssembler(
    inputCols=["sepal_length", "sepal_width", "petal_length", "petal_width"],
    outputCol="features"
)

In [ ]:
# Inicializamos el Modelo
lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=100)

In [ ]:

# Iniciamos nuestro pipeline y separamos en un set de entrenamiento y otro de pruebas.
pipeline = Pipeline(stages=[label_indexer, assembler, lr])
train, test = df.randomSplit([0.8, 0.2], seed=42)
model = pipeline.fit(train)


¿Notas cómo lo metimos dentro de un Pipeline? Esto nos ahorra tiempo, ya que de lo contrario tendríamos que estar dándole a un `.fit()` para cada parte del proceso.

In [ ]:
# 5) Evaluamos el modelo
pred = model.transform(test)
acc = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy"
).evaluate(pred)

print(f"Test accuracy: {acc:.3f}")

pred.select("features", "prediction", "probability").show(5, truncate=False)

Test accuracy: 1.000
+-----------------+----------+---------------------------------------------------+
|features         |prediction|probability                                        |
+-----------------+----------+---------------------------------------------------+
|[4.4,3.0,1.3,0.2]|1.0       |[3.2846336132346933E-31,1.0,1.1713089090333092E-55]|
|[4.6,3.2,1.4,0.2]|1.0       |[7.393509334952613E-34,1.0,1.5479602504474857E-58] |
|[4.6,3.6,1.0,0.2]|1.0       |[2.5273919968001947E-45,1.0,1.3168692004627705E-72]|
|[4.8,3.1,1.6,0.2]|1.0       |[3.8338070097546536E-29,1.0,6.765687668972633E-53] |
|[4.9,3.1,1.5,0.2]|1.0       |[3.8004464378364086E-29,1.0,2.327617171474784E-53] |
+-----------------+----------+---------------------------------------------------+
only showing top 5 rows



## ¿Y por qué no usar mejor Scikit-learn?

La principal diferencia es que Scikit-learn **no** es compatible con Dataframes de Spark. Es necesario regresar a un Dataframe de Pandas, o un arreglo de Numpy, y esas transformaciones de formatos, como recordarás, son muy pesadas.

Otra **gran** diferencia es que, siendo Scikit-learn más antiguo, tiene ya programada una mayor variedad de algoritmos, con mejor optimización de los mismos y más posibilidades de ajustar hiperparámetros.

Por último, aunque algunos métodos y funciones son similares, la mayoría cambia en su sintáxis.

### Tarea

Investiga algunas formas en las que podemos hacer Feature Engineering usando `spark.ml`. Por ejemplo, ¿cómo creo variables dummy a través de variables categóricas? ¿Cómo estandarizo mis datos?